In [1]:
!pip install mlconfig

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import copy
import os
from tqdm import tqdm
import torch.nn as nn
import torch
from torch.utils.data import Dataset
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 32
momentum=0.9
lr = 1e-3

cuda:0


In [3]:
import torch, torchvision
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split

import os, shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import mlconfig
from importlib import import_module
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def load_function(attr):
    module_, func = attr.rsplit('.', maxsplit=1)
    return getattr(import_module(module_), func)

config = mlconfig.load('config_resnet18.yaml')

In [4]:
from pathlib import Path

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import os
from tqdm import tqdm
import numpy as np

import torchvision.transforms as v1
from torchvision.io import read_image

plt.rcParams["savefig.bbox"] = 'tight'
torch.manual_seed(1)

In [5]:
class MyDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        csv = pd.read_csv(csv_file)
        self.sig_filenames = csv["fragment_name"].values
        self.sig_labels = csv["label"].values
        self.root_dir = root_dir

    def __len__(self):
        return len(self.sig_labels)

    def __getitem__(self, idx):
        #with open(self.root_dir + '/' + self.sig_filenames[idx] + '.npy', 'r+') as fl:
         # data_file = np.frombuffer(fl.read(), dtype=np.float32)
        data_file = np.asarray(np.load(self.root_dir + '/' + self.sig_filenames[idx] + '.npy', 'r+'))
        return data_file, self.sig_labels[idx]

In [6]:
dataset = MyDataset(root_dir='S:/drone_dataset_v4.0_pictures_spectr', csv_file='S:/drone_dataset_v4.0_pictures_spectr/dataset.csv')
train_set, valid_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
batch_size = 4
train_dataloader = torch.utils.data.DataLoader(
  train_set,
  batch_size=batch_size,
  shuffle=True,
  drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(
  valid_set,
  batch_size=batch_size,
  drop_last=True,
  shuffle=True)

In [7]:
def train_model(model, criterion, optimizer, num_epochs=20):
    train_losses = []
    val_losses = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                print(inputs.shape)
                print(labels)
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    print(outputs.dtype)
                    preds = outputs.max(dim=1, keepdim=True,)[1]
                    preds = torch.squeeze(preds)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val':
                val_losses.append(epoch_loss)
                if epoch_acc >= best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    
            if phase == 'train':
                train_losses.append(epoch_loss)
                 

        print()
 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, best_acc, train_losses, val_losses

In [8]:
dataloaders = {}
dataloaders['train'] = train_dataloader
dataloaders['val'] = valid_dataloader
dataset_sizes = {}
dataset_sizes['train'] = len(train_set)
dataset_sizes['val'] = len(valid_set)
print(dict(dataloaders))

{'train': <torch.utils.data.dataloader.DataLoader object at 0x000001376E05BA60>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x000001376E05B1F0>}


In [9]:
torch.cuda.empty_cache()
model = load_function(config.model.architecture)(pretrained=True)
model.fc = torch.nn.Linear(in_features=512, out_features=3, bias=True)
model.type(torch.cuda.FloatTensor)
model = model.to(device)

C:\Users\Admin\.conda\envs\python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\.conda\envs\python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 42.678MB


In [11]:
from tqdm import tqdm

optimizer = load_function(config.optimizer.name)(model.parameters(), lr=config.optimizer.lr)
criterion = load_function(config.loss_function.name)()

best_model, best_acc, train_losses, val_losses = train_model(model, criterion, optimizer, num_epochs=20)
name = 'model2.pth'
model.load_state_dict(best_model)
model = model.to('cpu')
torch.save(model.state_dict(), name)

Epoch 1/20
----------


  0%|                                                                                         | 0/2289 [00:00<?, ?it/s]

torch.Size([4, 3, 1024, 1024])
tensor([3, 3, 3, 3])
torch.float32


  0%|                                                                                         | 0/2289 [00:10<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
